In [3]:
import numpy as np
import warnings 
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/enb.arff')
data = pd.DataFrame(data)
data.head()

,Relative_compactness,X1,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,0.0,20.84,28.28


In [5]:
X = data.iloc[:,:-2].values
y = data.iloc[:,-2:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

(768, 8) (768, 2)


In [6]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(614, 8) (154, 8) (614, 2) (154, 2)


In [7]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print('MAE - ',mean_absolute_error(y_train,y_test))
    print('RMSE - ',mean_squared_error(y_train,y_test)**0.5)
    print('R-squared - ',r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 768
    p = 8

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

Using TensorFlow backend.


In [9]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

In [10]:
from time import time

In [11]:
chain = []

X_feed = X_train.copy()
t1 = time()

for i in range(y_train.shape[1]):
    print('Training chain node ',i)
    y_now = y_train[:,[i,]].copy()
    print('Shapes\n X = {} \n y = {}'.format(X_feed.shape,y_now.shape))
    
    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=500,batch_size=50)
    print('Training of node {} complete\n'.format(i))
    chain.append(node)
    X_feed = np.append(X_feed,y_now,axis=1)
    
t2 = time()

print('Time taken ',(t2-t1))

Training chain node  0
Shapes
 X = (614, 8) 
 y = (614, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
614/614 [==============================] - 0s - loss: 0.2458 - mean_squared_error: 0.2458     
Epoch 2/500
614/614 [==============================] - 0s - loss: 0.1190 - mean_squared_error: 0.1190     
Epoch 3/500
614/614 [==============================] - 0s - loss: 0.0870 - mean_squared_error: 0.0870     
Epoch 4/500
614/614 [==============================] - 0s - loss: 0.0769 - mean_squared_error: 0.0769     
Epoch 5/500
614/614 [==============================] - 0s - loss: 0.0710 - mean_squared_error: 0.0710     
Epoch 6/500
614/614 [==============================] - 0s - loss: 0.0634 - mean_squared_error: 0.0634     
Epoch 7/500
614/614 [==============================] - 0s - loss: 0.0597 - mean_squared_error: 0.0597     
Epoch 8/500
614/614 [==============================] - 0s - loss: 0.

614/614 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 63/500
614/614 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024     
Epoch 64/500
614/614 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 65/500
614/614 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 66/500
614/614 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 67/500
614/614 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 68/500
614/614 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 69/500
614/614 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 70/500
614/614 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 71/500
614/614 [===================

614/614 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 135/500
614/614 [==============================] - 0s - loss: 9.0493e-04 - mean_squared_error: 9.0493e-04
Epoch 136/500
614/614 [==============================] - 0s - loss: 6.6805e-04 - mean_squared_error: 6.6805e-04     
Epoch 137/500
614/614 [==============================] - 0s - loss: 5.9998e-04 - mean_squared_error: 5.9998e-04     
Epoch 138/500
614/614 [==============================] - 0s - loss: 7.4071e-04 - mean_squared_error: 7.4071e-04     
Epoch 139/500
614/614 [==============================] - 0s - loss: 7.3242e-04 - mean_squared_error: 7.3242e-04     
Epoch 140/500
614/614 [==============================] - 0s - loss: 8.8622e-04 - mean_squared_error: 8.8622e-04
Epoch 141/500
614/614 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
Epoch 142/500
614/614 [==============================] - 0s - loss: 9.4859e-04 - mean_squared_er

614/614 [==============================] - 0s - loss: 6.8384e-04 - mean_squared_error: 6.8384e-04     
Epoch 206/500
614/614 [==============================] - 0s - loss: 4.7920e-04 - mean_squared_error: 4.7920e-04     
Epoch 207/500
614/614 [==============================] - 0s - loss: 4.7546e-04 - mean_squared_error: 4.7546e-04     
Epoch 208/500
614/614 [==============================] - 0s - loss: 6.7134e-04 - mean_squared_error: 6.7134e-04     
Epoch 209/500
614/614 [==============================] - 0s - loss: 7.0066e-04 - mean_squared_error: 7.0066e-04     
Epoch 210/500
614/614 [==============================] - 0s - loss: 6.0544e-04 - mean_squared_error: 6.0544e-04     
Epoch 211/500
614/614 [==============================] - 0s - loss: 7.9685e-04 - mean_squared_error: 7.9685e-04
Epoch 212/500
614/614 [==============================] - 0s - loss: 4.4061e-04 - mean_squared_error: 4.4061e-04     
Epoch 213/500
614/614 [==============================] - 0s - loss: 3.7754e-04 - me

614/614 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024     
Epoch 277/500
614/614 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 278/500
614/614 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014     
Epoch 279/500
614/614 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013             
Epoch 280/500
614/614 [==============================] - 0s - loss: 8.9331e-04 - mean_squared_error: 8.9331e-04     
Epoch 281/500
614/614 [==============================] - 0s - loss: 8.3932e-04 - mean_squared_error: 8.3932e-04     
Epoch 282/500
614/614 [==============================] - 0s - loss: 6.0523e-04 - mean_squared_error: 6.0523e-04     
Epoch 283/500
614/614 [==============================] - 0s - loss: 4.9138e-04 - mean_squared_error: 4.9138e-04     
Epoch 284/500
614/614 [==============================] - 0s - loss: 4.3122e-04 - mean_squared_error: 4

...

614/614 [==============================] - 0s - loss: 8.0500e-04 - mean_squared_error: 8.0500e-04     
Epoch 207/500
614/614 [==============================] - 0s - loss: 6.6274e-04 - mean_squared_error: 6.6274e-04
Epoch 208/500
614/614 [==============================] - 0s - loss: 6.4913e-04 - mean_squared_error: 6.4913e-04     
Epoch 209/500
614/614 [==============================] - 0s - loss: 8.6089e-04 - mean_squared_error: 8.6089e-04
Epoch 210/500
614/614 [==============================] - 0s - loss: 7.4254e-04 - mean_squared_error: 7.4254e-04     
Epoch 211/500
614/614 [==============================] - 0s - loss: 6.8352e-04 - mean_squared_error: 6.8352e-04     
Epoch 212/500
614/614 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010             
Epoch 213/500
614/614 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011     
Epoch 214/500
614/614 [==============================] - 0s - loss: 9.2599e-04 - mean_squared_er

614/614 [==============================] - 0s - loss: 5.6261e-04 - mean_squared_error: 5.6261e-04     
Epoch 278/500
614/614 [==============================] - 0s - loss: 7.3280e-04 - mean_squared_error: 7.3280e-04     
Epoch 279/500
614/614 [==============================] - 0s - loss: 7.8335e-04 - mean_squared_error: 7.8335e-04     
Epoch 280/500
614/614 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
Epoch 281/500
614/614 [==============================] - 0s - loss: 9.0964e-04 - mean_squared_error: 9.0964e-04
Epoch 282/500
614/614 [==============================] - 0s - loss: 8.4287e-04 - mean_squared_error: 8.4287e-04
Epoch 283/500
614/614 [==============================] - 0s - loss: 7.5253e-04 - mean_squared_error: 7.5253e-04     
Epoch 284/500
614/614 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
Epoch 285/500
614/614 [==============================] - 0s - loss: 8.4545e-04 - mean_sq

614/614 [==============================] - 0s - loss: 5.4773e-04 - mean_squared_error: 5.4773e-04     
Epoch 349/500
614/614 [==============================] - 0s - loss: 5.2071e-04 - mean_squared_error: 5.2071e-04     
Epoch 350/500
614/614 [==============================] - 0s - loss: 5.4085e-04 - mean_squared_error: 5.4085e-04     
Epoch 351/500
614/614 [==============================] - 0s - loss: 3.8745e-04 - mean_squared_error: 3.8745e-04     
Epoch 352/500
614/614 [==============================] - 0s - loss: 3.4623e-04 - mean_squared_error: 3.4623e-04     
Epoch 353/500
614/614 [==============================] - 0s - loss: 5.6090e-04 - mean_squared_error: 5.6090e-04     
Epoch 354/500
614/614 [==============================] - 0s - loss: 6.3386e-04 - mean_squared_error: 6.3386e-04     
Epoch 355/500
614/614 [==============================] - 0s - loss: 5.8187e-04 - mean_squared_error: 5.8187e-04     
Epoch 356/500
614/614 [==============================] - 0s - loss: 4.3000e-04

614/614 [==============================] - 0s - loss: 6.2728e-04 - mean_squared_error: 6.2728e-04     
Epoch 419/500
614/614 [==============================] - 0s - loss: 5.9706e-04 - mean_squared_error: 5.9706e-04     
Epoch 420/500
614/614 [==============================] - 0s - loss: 8.1517e-04 - mean_squared_error: 8.1517e-04     
Epoch 421/500
614/614 [==============================] - 0s - loss: 7.9142e-04 - mean_squared_error: 7.9142e-04     
Epoch 422/500
614/614 [==============================] - 0s - loss: 8.5465e-04 - mean_squared_error: 8.5465e-04     
Epoch 423/500
614/614 [==============================] - 0s - loss: 7.7724e-04 - mean_squared_error: 7.7724e-04
Epoch 424/500
614/614 [==============================] - 0s - loss: 6.0781e-04 - mean_squared_error: 6.0781e-04     
Epoch 425/500
614/614 [==============================] - 0s - loss: 7.2991e-04 - mean_squared_error: 7.2991e-04     
Epoch 426/500
614/614 [==============================] - 0s - loss: 7.9924e-04 - me

614/614 [==============================] - 0s - loss: 1.6476e-04 - mean_squared_error: 1.6476e-04     
Epoch 490/500
614/614 [==============================] - 0s - loss: 1.4696e-04 - mean_squared_error: 1.4696e-04     
Epoch 491/500
614/614 [==============================] - 0s - loss: 1.4400e-04 - mean_squared_error: 1.4400e-04     
Epoch 492/500
614/614 [==============================] - 0s - loss: 1.1888e-04 - mean_squared_error: 1.1888e-04     
Epoch 493/500
614/614 [==============================] - 0s - loss: 1.5369e-04 - mean_squared_error: 1.5369e-04     
Epoch 494/500
614/614 [==============================] - 0s - loss: 2.5377e-04 - mean_squared_error: 2.5377e-04     
Epoch 495/500
614/614 [==============================] - 0s - loss: 3.2790e-04 - mean_squared_error: 3.2790e-04     
Epoch 496/500
614/614 [==============================] - 0s - loss: 3.3198e-04 - mean_squared_error: 3.3198e-04     
Epoch 497/500
614/614 [==============================] - 0s - loss: 3.5680e-04

In [12]:
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print('getting output from chain node ',i)
    node_name = 'Node' + str(i)
    print('Shapes - \n X = {}'.format(X_feed.shape))
    output = node.predict(X_feed)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i += 1

getting output from chain node  0
Shapes - 
 X = (154, 8)
getting output from chain node  1
Shapes - 
 X = (154, 9)


In [13]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [14]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

MAE -  0.8262223137199108
RMSE -  1.1025508670658426
R-squared -  0.9873102652236903
Adjusted R-squared -  0.9871765130784854
